In [ ]:
!pip -q install tqdm Sentencepiece pandas torchtext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.2 MB/s eta 0:00:00


In [ ]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset,Dataset,DatasetDict
import os
import pandas as pd
import tqdm as tqdm
import ast
import time

In [ ]:

# Load data set from huggingface
data_sample = load_dataset("QuyenAnhDE/Diseases_Symptoms")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/381 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


Diseases_Symptoms.csv:   0%|          | 0.00/107k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [ ]:
updated_data = [{'Name': item['Name'], 'Symptoms': item['Symptoms']} for item in data_sample['train']]


In [ ]:
df=pd.DataFrame(updated_data)

In [ ]:
df.head(5)

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [ ]:

# Just extract the Symptoms
df['Symptoms'] = df['Symptoms'].apply(lambda x: ', '.join(x.split(', ')))
display(df.head())

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [ ]:
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from torch.utils.data import Dataset,DataLoader,random_split

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
model=AutoModelForCausalLM.from_pretrained("distilgpt2")

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
tokenizer=AutoTokenizer.from_pretrained("distilgpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
if torch.cuda.is_available():

  device=torch.device("cuda")
else:

  device=torch.device("cpu")

In [ ]:
device

device(type='cuda')

In [ ]:
df.describe()

,Name,Symptoms
count,400,400
unique,392,395
top,Sciatica,"Swelling, pain, dry mouth, bad taste"
freq,3,3


In [ ]:
batch_size=8

In [ ]:
# Dataset Prep
class LanguageDataset(Dataset):
    """
    An extension of the Dataset object to:
      - Make training loop cleaner
      - Make ingestion easier from pandas df's
    """
    def __init__(self, df, tokenizer):
        self.labels = df.columns
        self.data = df.to_dict(orient='records')
        self.tokenizer = tokenizer
        x = self.fittest_max_length(df)  # Fix here
        self.max_length = x

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx][self.labels[0]]
        y = self.data[idx][self.labels[1]]
        text = f"{x} | {y}"
        tokens = self.tokenizer.encode_plus(text, return_tensors='pt', max_length=128, padding='max_length', truncation=True)
        return tokens

    def fittest_max_length(self, df):  # Fix here
        """
        Smallest power of two larger than the longest term in the data set.
        Important to set up max length to speed training time.
        """
        max_length = max(len(max(df[self.labels[0]], key=len)), len(max(df[self.labels[1]], key=len)))
        x = 2
        while x < max_length: x = x * 2
        return x

# Cast the Huggingface data set as a LanguageDataset we defined above
data_sample = LanguageDataset(df, tokenizer)


In [ ]:
data_sample

In [ ]:
#train and validation split
train_size=int(0.8*len(data_sample))
val_size=len(data_sample)-train_size

In [ ]:
train_data,validation_data=random_split(data_sample,[train_size,val_size])

In [ ]:
#
# Make the iterators
train_loader=DataLoader(train_data,batch_size=8,shuffle=True)
val_loader=DataLoader(validation_data,batch_size=8)

In [ ]:
train_loader

In [ ]:
num_epochs = 10
batch_size = 8
model_name = 'distilgpt2'
gpu = 0

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side="right"
criterion =nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)


In [ ]:
result=pd.DataFrame(columns=["epoch","transformer","batch_Size","gpu","training_loss","validation_loss","epoch_duration_sec"])

In [ ]:
from transformers import Trainer, TrainingArguments


In [ ]:
training_arguments=TrainingArguments(
    output_dir="./result"
    num_train_epochs=num_epochs,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=3,
    weight_decay=0.01,
    logging_dir='./logs'


)

In [ ]:
trainer = Trainer(
    model=model,
    train_loader=train_loader, val_loader=val_loader, criterion=criterion, optimizer=optimizer
)

TypeError: Trainer.__init__() got an unexpected keyword argument 'train_loader'

In [ ]:
from transformers import Trainer, TrainingArguments

# Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",  # Corrected output directory name
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,  # Use batch_size variable
    per_device_eval_batch_size=batch_size,   # Use batch_size variable
    warmup_steps=3,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch"  # Evaluate at the end of each epoch
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,  # Pass TrainingArguments
    train_dataset=train_data,  # Pass train_data as a Dataset object
    eval_dataset=validation_data,  # Pass validation_data as a Dataset object
    #criterion=criterion,  # Remove criterion and optimizer - handled by Trainer
    #optimizer=optimizer   # Remove criterion and optimizer - handled by Trainer
)

# Train the model
trainer.train()

In [ ]:
trainer.train(num_epochs=10)

#now

In [ ]:
data_sample = load_dataset("QuyenAnhDE/Diseases_Symptoms")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/381 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


Diseases_Symptoms.csv:   0%|          | 0.00/107k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [ ]:
dataset=pd.DataFrame(data_sample['train'])

In [ ]:
dataset=dataset.drop(columns=['Code'])

In [ ]:
dataset

,Name,Symptoms,Treatments
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o...","Antidepressant medications, Cognitive Behavior..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue","Voice Rest, Speech Therapy, Surgical Removal"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck...","Growth hormone therapy, Estrogen replacement t..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro...",Observation and monitoring (in cases of mild o...
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala...","Supportive Measures, Gastric Decontamination, ..."
...,...,...,...
395,Urinary Stones (Kidney Stones),"Severe abdominal or back pain, blood in urine,...","Pain management, increased fluid intake, medic..."
396,Osteoporosis,"Fragile bones, loss of height over time, back ...","Calcium and vitamin D supplements, regular exe..."
397,Rheumatoid Arthritis,"Joint pain, stiffness, swelling, fatigue, loss...",Medications (nonsteroidal anti-inflammatory dr...
398,Type 1 Diabetes,"Frequent urination, Increased thirst, Weight loss","Insulin therapy, Blood sugar monitoring, Healt..."


In [ ]:
from sklearn.model_selection import train_test_split
train_data,val_data=train_test_split(dataset,test_size=0.2,random_state=42)

In [ ]:
from transformers import AutoTokenizer,AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"
special_token = " | "  # Choose a special token
model=AutoModelForCausalLM.from_pretrained("distilgpt2")

In [ ]:
import torch
from torch.utils.data import Dataset,DataLoader
class DiseaseSymptomDataset(Dataset):
  def __init__(self,data,tokenizer,special_token):
    self.data=data
    self.tokenizer=tokenizer
    self.special_token=special_token
    self.max_length=self.fittest_max_length(data)  # Calling the method with data
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    disease=self.data.iloc[idx]['Name']
    sysmtoms=self.data.iloc[idx]['Symptoms']
    Treatments=self.data.iloc[idx]['Treatments']
    text=f"{disease}{self.special_token}{sysmtoms}{self.special_token}{Treatments}"
    tokens=self.tokenizer.encode_plus(text,return_tensors="pt",max_length=self.max_length,padding='max_length',truncation=True)  # Using 'max_length' for padding
    return tokens
  def fittest_max_length(self, df):  # Added self parameter
    """
    Smallest power of two larger than the longest term in the data set.
    Important to set up max length to speed training time.
    """
    # Handle None values in 'Treatments' column
    df['Treatments'] = df['Treatments'].astype(str)  # Convert to string type


    max_length = max(
        len(max(df['Name'], key=len)),
        len(max(df['Symptoms'], key=len)),
        len(max(df['Treatments'],key=len))
    )
    x = 2
    while x < max_length:
        x = x * 2
    return x

In [ ]:
train_dataset = DiseaseSymptomDataset(train_data, tokenizer, special_token)
eval_dataset = DiseaseSymptomDataset(val_data, tokenizer, special_token)

In [ ]:
batch_size=8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
train_loader

In [ ]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, EarlyStoppingCallback
import torch

# ... (rest of your code) ...

def compute_loss(model, inputs, return_outputs=False):
    """
    Custom compute_loss function for the Trainer.

    Args:
        model: The model being trained.
        inputs: The input data.
        return_outputs: Whether to return the model outputs.

    Returns:
        The computed loss.
    """
    labels = inputs.get("input_ids")  # Assuming labels are the same as input_ids
    # Shift labels to the left by one position
    labels = torch.roll(labels, shifts=-1, dims=1)
    # Replace the last token of each sequence with the pad token
    labels[:, -1] = tokenizer.pad_token_id

    outputs = model(**inputs, labels=labels)  # Pass labels to the model

    # Extract the loss from the outputs
    loss = outputs.loss
    # If return_outputs is True, return both the loss and the outputs
    if return_outputs:
        return (loss, outputs)
    # Otherwise, just return the loss
    return loss

# ... (rest of your code) ...

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    compute_loss=compute_loss # Pass the custom compute_loss function to the Trainer

)

# ... (rest of your code) ...

TypeError: Trainer.__init__() got an unexpected keyword argument 'compute_loss'

In [ ]:
# Fine-tuning the Model
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments,EarlyStoppingCallback

training_args=TrainingArguments(
    output_dir="./result",
    logging_dir="./logs",
    logging_steps=50,
    logging_strategy="steps",
    eval_strategy ="steps",
    eval_steps=50,

    learning_rate=5e-4,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=12,
    max_steps=1000,

    warmup_steps=int(0.1*1000),

    weight_decay=0.01,
    save_strategy="steps",
    save_steps=50,
   # Added learning_rate
    fp16=True,
    gradient_accumulation_steps=2,
    torch_empty_cache_steps=50,
    report_to="none",
    load_best_model_at_end=True,  # Enable loading the best model
    metric_for_best_model="loss"  # Specify the metric for se






)


In [ ]:
import torch
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    compute_loss_func =torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

)

In [ ]:
import time

start_time = time.time()  # Record the start time

trainer.train()  # Start the training process

end_time = time.time()  # Record the end time
total_time = end_time - start_time  # Calculate the total time

print(f"Total training time: {total_time:.2f} seconds")  # Print the total time

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [ ]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, EarlyStoppingCallback
import torch

# ... (rest of your code) ...

def compute_loss(model, inputs, return_outputs=False):
    """
    Custom compute_loss function for the Trainer.

    Args:
        model: The model being trained.
        inputs: The input data.
        return_outputs: Whether to return the model outputs.

    Returns:
        The computed loss.
    """
    labels = inputs.get("input_ids")  # Assuming labels are the same as input_ids
    # Shift labels to the left by one position
    labels = torch.roll(labels, shifts=-1, dims=1)
    # Replace the last token of each sequence with the pad token
    labels[:, -1] = tokenizer.pad_token_id
    # Remove the first token of the labels (as it's the start token)
    labels = labels[:, 1:]
    # Remove the last token of the inputs (as it doesn't have a corresponding label)
    inputs["input_ids"] = inputs["input_ids"][:, :-1]
    inputs["attention_mask"] = inputs["attention_mask"][:, :-1]
    # Pass the inputs through the model
    outputs = model(**inputs, labels=labels)

    # Extract the loss from the outputs
    loss = outputs.loss
    # If return_outputs is True, return both the loss and the outputs
    if return_outputs:
        return (loss, outputs)
    # Otherwise, just return the loss
    return loss

# ... (rest of your code) ...

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    compute_loss_func=compute_loss # Pass the custom compute_loss function to the Trainer

)

# ... (rest of your code) ...